In [27]:
import pandas as pd
import numpy as np

In [28]:
from typing import List, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
songs = pd.read_csv("songdata.csv")
songs.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [30]:
songs = songs.sample(n=5000).drop("link", axis=1).reset_index(drop=True)

In [31]:
songs["text"] = songs["text"].str.replace(r"\n", "")

In [32]:
tfidf = TfidfVectorizer(analyzer="word", stop_words="english")

In [33]:
lyrics_matrix = tfidf.fit_transform(songs["text"])

In [34]:
cosine_similarities = cosine_similarity(lyrics_matrix)

In [35]:
similarities = {}

In [36]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs.
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]


In [37]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)

        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score")
            print("--------------------")

    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [38]:
recommedations = ContentBasedRecommender(similarities)

In [39]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4
}

In [40]:
recommedations.recommend(recommendation)

The 4 recommended songs for I Ain't No Angel are:
Number 1:
If You Be My Baby by Fleetwood Mac with 0.272 similarity score
--------------------
Number 2:
Angel Baby by Linda Ronstadt with 0.267 similarity score
--------------------
Number 3:
Angel On My Shoulder by Natalie Cole with 0.266 similarity score
--------------------
Number 4:
Angel Eyes by Ace Of Base with 0.253 similarity score
--------------------


In [41]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4
}

In [42]:
recommedations.recommend(recommendation2)

The 4 recommended songs for Ddevil are:
Number 1:
All God's Lovely Children by Kenny Rogers with 0.228 similarity score
--------------------
Number 2:
Too Many People by Pet Shop Boys with 0.196 similarity score
--------------------
Number 3:
People by King Crimson with 0.19 similarity score
--------------------
Number 4:
Some People Are Crazy by John Martyn with 0.162 similarity score
--------------------


In [43]:
import pickle


pickle.dump(similarities, open("similarities.pkl", "wb"))

pickle.dump(songs, open("songs.pkl", "wb"))